## Multivaraite Datenanalyse
# Lineare Diskriminanzanalyse (LDA)
### Michael Araz, Daniel Hasenklever, Stefan Pede

In [3]:
import numpy as np
import os
import pandas as pd

mmPfad = r"D:\C\Uni\Master\KSS\MV_Analyse\Python\Rep\tools\Messmatrix_Beispiel_small.csv"

### Eigenschaften der Messmatrix 

In [4]:
df = pd.read_csv(mmPfad)
print("Anzahl der Kennwerte: "+str(df.shape[1]))
print("Anzahl der vermessenen Rohre: "+str(df.shape[0]))
print("Anzahl der gefahrenen Produkte: "+str(df.groupby(["Header_Leitguete","Header_Soll_AD","Header_Soll_WD"])["Header_Pseudonummer"].agg(["count"]).shape[0]))
print("Anzahl der Walzlose: "+str(len(pd.unique(df["Header_Walzlos"]))))
print("\nAuszug:")
df.head()

Anzahl der Kennwerte: 74
Anzahl der vermessenen Rohre: 10000
Anzahl der gefahrenen Produkte: 24
Anzahl der Walzlose: 145

Auszug:


,Header_Pseudonummer,Header_Walzlos,Header_Leitguete,Header_Anzahl_Bloecke_im_Los,Header_Soll_AD,Header_Soll_WD,WAS_Blockmasse,WAS_Blocklaenge,WAS_Kerntemperatur,WAS_Oberflaechentemperatur,...,SRW_Endwalztemp_Mittelwert,SRW_Endwalztemp_Kruemmung,Exzentri_Mittelwert,Exzentri_MW_Anfang,Exzentri_MW_Mitte,Exzentri_MW_Ende,Exzentri_Standardabweichung,Exzentri_Maximum,SPL_T_vor_SPL_Mittelwert,SPL_T_vor_SPL_Kruemmung
0,1.283698e+09,71600914.0,10300.0,177.0,31.8,3.6,201.0,730.8,650.0,712.0,...,NaN,NaN,1.312685,1.642733,1.107086,1.396027,0.678052,4.1088,NaN,NaN
1,1.283699e+09,71600914.0,10300.0,177.0,31.8,3.6,201.2,731.1,650.0,691.0,...,NaN,NaN,1.806910,1.717785,1.624160,2.253463,0.748043,5.0915,NaN,NaN
2,1.283699e+09,71600914.0,10300.0,177.0,31.8,3.6,201.2,732.0,650.0,723.0,...,NaN,NaN,2.358595,2.804569,1.781053,3.097531,1.114924,5.8935,NaN,NaN
3,1.283700e+09,71600914.0,10300.0,177.0,31.8,3.6,200.6,730.8,650.0,727.0,...,NaN,NaN,1.901708,2.404824,1.767721,1.761576,1.060964,7.3769,NaN,NaN
4,1.283700e+09,71600914.0,10300.0,177.0,31.8,3.6,200.6,730.8,650.0,722.0,...,NaN,NaN,1.462318,1.449374,1.060459,2.271651,0.799694,7.0515,NaN,NaN


### Produkte

In [5]:
df.groupby(["Header_Leitguete","Header_Soll_AD","Header_Soll_WD"])["Header_Pseudonummer"].agg(["count"])

count
Header_Leitguete Header_Soll_AD Header_Soll_WD       
10300.0          21.3           2.50              167
                 23.0           2.60              156
                 25.0           2.40              214
                                2.55              755
                                2.80             1335
                                3.60              414
                 28.0           2.60              279
                 31.8           3.20              190
                                3.60              686
                 42.4           2.60              336
                 48.3           2.60              150
12400.0          35.0           2.60               70
                 42.4           3.60              114
13100.0          21.3           3.00              174
40200.0          30.0           2.70              806
                 36.0           3.20               70
                 38.0           2.60              573
                 42.4           3.20              159
41100.0          35.0           2.70              633
90000.0          25.0           2.20              129
                                2.55              381
                 30.0           2.75              857
                 31.8           2.60              323
94800.0          31.8           3.30             1029

### Vorverarbeiten der Kennwerte

In [8]:
relNaNsCol = np.sum(np.isnan(df))/df.shape[0]*100
# schmeiße zunächst alle Spalten heraus, die mehr als bestimmte Prozent an NaNs haben
spaltenSchranke = 15 # % der NaNs in Spalte
keep = [i for i in np.arange(len(relNaNsCol)) if relNaNsCol[i] <= spaltenSchranke]
dfVV = df[df.columns[keep]] # extrahiere Spalten

# gleiches auf Zeilen anwenden
zeilenSchranke = 5 # % der NaNs in Zeile
relNaNsRow = dfVV.isnull().sum(axis=1)/dfVV.shape[1]*100
keep = [i for i in np.arange(len(relNaNsRow)) if relNaNsRow[i] <= zeilenSchranke]
dfVV2 = dfVV.iloc[keep] #extraheire Zeilen

#übrige NaNs mit Mittelwert aus Spalten auffüllen
dfVV2.fillna(dfVV2.mean())

# Ausgabe
print("Daten nach Vorverarbeitung:")
print("Anzahl der Kennwerte: "+str(dfVV2.shape[1]))
print("Anzahl der vermessenen Rohre: "+str(dfVV2.shape[0]))
print("Anzahl der gefahrenen Produkte: "+str(dfVV2.groupby(["Header_Leitguete","Header_Soll_AD","Header_Soll_WD"])["Header_Pseudonummer"].agg(["count"]).shape[0]))
print("Anzahl der Walzlose: "+str(len(pd.unique(dfVV2["Header_Walzlos"]))))
print("\nAuszug:")
dfVV2.head()

Daten nach Vorverarbeitung:
Anzahl der Kennwerte: 45
Anzahl der vermessenen Rohre: 8878
Anzahl der gefahrenen Produkte: 24
Anzahl der Walzlose: 138

Auszug:


,Header_Pseudonummer,Header_Walzlos,Header_Leitguete,Header_Anzahl_Bloecke_im_Los,Header_Soll_AD,Header_Soll_WD,WAS_Blockmasse,WAS_Blocklaenge,WAS_Kerntemperatur,WAS_Oberflaechentemperatur,...,LOP_std_dev,LOP_Kruemmung,LOP_Stoesselweg,LOP_Geschw,Exzentri_Mittelwert,Exzentri_MW_Anfang,Exzentri_MW_Mitte,Exzentri_MW_Ende,Exzentri_Standardabweichung,Exzentri_Maximum
33,1.283714e+09,71600914.0,10300.0,177.0,31.8,3.6,200.6,731.1,650.0,700.0,...,96.148627,-2203.180151,0.8150,0.004497,2.917560,3.186888,2.064545,4.354145,1.458293,8.7595
34,1.283716e+09,71600914.0,10300.0,177.0,31.8,3.6,201.0,731.0,650.0,742.0,...,177.305887,-2775.029343,0.8490,0.004437,2.001135,2.336132,1.475950,2.738891,1.035995,6.0952
35,1.283716e+09,71600914.0,10300.0,177.0,31.8,3.6,200.6,731.2,650.0,718.0,...,129.428154,-1934.765965,0.8090,0.004538,1.486782,1.491319,1.162636,2.135003,0.975517,9.0817
36,1.283717e+09,71600914.0,10300.0,177.0,31.8,3.6,201.6,731.6,650.0,721.0,...,86.090037,-1828.211936,0.8170,0.004539,1.368778,1.507379,1.187876,1.593181,0.709784,6.0057
37,1.283718e+09,71600914.0,10300.0,177.0,31.8,3.6,200.4,730.9,650.0,719.0,...,147.809998,-2617.471836,0.8185,0.004490,1.248656,1.108136,0.826779,2.199452,0.802720,4.8040
